In [1]:
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:reporting-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
%use dataframe
%use kandy

In [2]:
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val userSdk = UserSdk()
val username = "Johann-8.1"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}

user

UserTO(id=190e8ce7-83b6-43fa-b1ee-303ab2147b42, username=Johann-8.1)

In [3]:
import ro.jf.funds.fund.sdk.FundSdk
import kotlinx.coroutines.runBlocking

val fundSdk = FundSdk()
val expenseFund = runBlocking {
    fundSdk.listFunds(user.id).items.find { it.name.value == "Expenses" }
}
expenseFund

FundTO(id=7bc9ded0-f031-44c7-9b4e-41b7f697e18c, name=Expenses)

In [4]:
import ro.jf.funds.reporting.sdk.ReportingSdk
import ro.jf.funds.reporting.api.model.*
import kotlinx.coroutines.delay
import ro.jf.funds.commons.model.labelsOf
import ro.jf.funds.commons.model.Currency
import java.time.Instant

val reportingSdk = ReportingSdk()
val reportView = runBlocking {
    val reportViewName = "Expense report"
    val existingReportView = reportingSdk.listReportViews(user.id).items.firstOrNull { it.name == reportViewName }
    if (existingReportView != null) {
        return@runBlocking existingReportView
    }
    val labels = labelsOf("basic", "home", "shopping_services", "transport", "fun", "gifts", "development")
    val request = CreateReportViewTO(reportViewName, expenseFund.id, ReportViewType.EXPENSE, Currency.RON, labels)
    var task: ReportViewTaskTO = reportingSdk.createReportView(user.id, request)
    val timeout = Instant.now().plusSeconds(120)
    while (task is ReportViewTaskTO.InProgress && Instant.now().isBefore(timeout)) {
        delay(2000)
        task = reportingSdk.getReportViewTask(user.id, task.taskId)
    }
    if (task is ReportViewTaskTO.Completed) {
        task.report
    } else {
        throw IllegalStateException("Report view creation timed out on task $task")
    }
}
reportView

ReportViewTO(id=756aac0c-3529-4f16-abdf-9ec3d95e6062, name=Expense report, fundId=7bc9ded0-f031-44c7-9b4e-41b7f697e18c, type=EXPENSE, currency=Currency(value=RON), labels=[basic, home, shopping_services, transport, fun, gifts, development])

In [5]:
import ro.jf.funds.reporting.api.model.*

val granularInterval = GranularDateInterval(
    interval = DateInterval(
        LocalDate.parse("2019-01-01"),
        LocalDate.parse("2019-12-31")
    ),
    granularity = TimeGranularity.MONTHLY
)
val report = runBlocking {
    reportingSdk.getReportViewData(user.id, reportView.id, granularInterval)
} as ExpenseReportDataTO
report

ExpenseReportDataTO(viewId=756aac0c-3529-4f16-abdf-9ec3d95e6062, granularInterval=GranularDateInterval(interval=DateInterval(from=2019-01-01, to=2019-12-31), granularity=MONTHLY), data=[DataItem(timeBucket=2019-01-01, amount=-4468.51), DataItem(timeBucket=2019-02-01, amount=-4301.32), DataItem(timeBucket=2019-03-01, amount=-9171.45), DataItem(timeBucket=2019-04-01, amount=-6967.76), DataItem(timeBucket=2019-05-01, amount=-9043.57), DataItem(timeBucket=2019-06-01, amount=-7024.09), DataItem(timeBucket=2019-07-01, amount=-11232.58), DataItem(timeBucket=2019-08-01, amount=-25034.218092), DataItem(timeBucket=2019-09-01, amount=0.0), DataItem(timeBucket=2019-10-01, amount=0.0), DataItem(timeBucket=2019-11-01, amount=0.0), DataItem(timeBucket=2019-12-01, amount=0.0)])

In [6]:
import java.math.BigDecimal

data class MonthlyExpense(val month: LocalDate, val amount: Double)

val monthlyReportDF = report.data
    .map {
        MonthlyExpense(it.timeBucket, it.amount.negate().toDouble())
    }
    .toDataFrame()

In [7]:
import org.jetbrains.kotlinx.kandy.dsl.* // For creating visualizations
import org.jetbrains.letsPlot.scale.scaleYContinuous

plot(monthlyReportDF) {
    y("amount")
    bars {
        x("month") {

        }
    }
    line {
        x("month") {

        }
        color = Color.hex("#6e5596")
    }
    layout {
        title = "Monthly expenses"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="DoexBG"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Monthly expenses"
},
"mapping":{
},
"data":{
"amount":[4468.51,4301.32,9171.45,6967.76,9043.57,7024.09,11232.58,25034.218092,0.0,0.0,0.0,0.0],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"amount",
"x":"month"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
"y":"amount",
"x":"month"
},
"stat":"identity",
"color":"#6e5596",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"column":"month",
"type":"datetime"
}]
}
};
 var plotContainer = document.getElementById("DoexBG");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M86.08523391008202 432.4319410091562 L86.08523391008202 432.4319410091562 L173.4099149052563 435.74572877793804 L252.2838203202482 339.2173644835584 L339.6085013154225 382.895580693779 L424.1162571172026 341.7520087796995 L511.4409381123769 381.77909247668777 L595.948693914157 298.36475736669 L683.273374909324 24.809523809523853 L770.5980559044983 521.0 L855.1058117062785 521.0 L942.4304927014527 521.0 L1026.938248503233 521.0 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(110,85,150)" stroke-opacity="1.0">
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 Jun 
 
 
 
 
 
 
 
 
 Jul 
 
 
 
 
 
 
 
 
 Aug 
 
 
 
 
 
 
 
 
 Sep 
 
 
 
 
 
 
 
 
 Oct 
 
 
 
 
 
 
 
 
 Nov 
 
 
 
 
 
 
 
 
 Dec 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 6,000 
 
 
 
 
 
 
 8,000 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 12,000 
 
 
 
 
 
 
 14,000 
 
 
 
 
 
 
 16,000 
 
 
 
 
 
 
 18,000 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 22,000 
 
 
 
 
 
 
 24,000 
 
 
 
 
 
 
 26,000 
 
 
 
 
 
 
 
 
 Monthly expenses 
 
 
 
 
 amount 
 
 
 
 
 month